# Genetic Algorithm TSP 

## Imports and Utilities

In [1]:
import time
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial
import copy
import operator
import pandas as pd
import numpy as np

def att48_to_nx3_list(filename):

    with open(filename) as f:
        att48 = f.readlines()

    #Rejects document text 
    pre_size = len(att48)
    att48 =att48[6:pre_size-1] 
    post_size = len(att48)
    
    #separates and splits numbers into list array[cityNUM, xcoord, ycoord]
    #split each line into 3 colums cityID, x_coord, y_coord
    for i in range (post_size):
        att48[i] = att48[i].split()
    
    #Convert all strings in array to ints 
    for i in range (post_size):
        att48[i][0] = int(att48[i][0])
        att48[i][1] = int(att48[i][1])
        att48[i][2] = int(att48[i][2])
         
    return att48

def plot_cities(cities_list):
    
    cityIDs = []
    x = []
    y = []
        
    for i in range (len(cities_list)):
        cityIDs.append(cities_list[i][0])
        x.append(cities_list[i][1])
        y.append(cities_list[i][2])
    
    plt.plot(x, y, 'k-')
    plt.show()


def read_optimum_tour_index(opt_file):
    
    with open(opt_file) as f:
        att48 = f.readlines()
        
    #Rejects document text 
    pre_size = len(att48)
    att48 =att48[5:pre_size-2] 
    post_size = len(att48)
    
    #separates and splits numbers into list array[cityNUM, xcoord, ycoord]
    
    #split each line into 3 colums cityID, x_coord, y_coord
    for i in range (post_size):
        att48[i] = att48[i].split()
    
    #Convert all strings in array to ints 
    for i in range (post_size):
        att48[i][0]= int(att48[i][0])
           
    return att48

def sort_to_index(cities, index):
    #intilize list
    sorted_cities = []
    
    #initilze list size
    for i in range (len(index)):
        sorted_cities.append([0,0,0])
        
    #rearrange city index to optimum solution
    for i in range (len(index)):
        sorted_cities[i][0] = cities[index[i][0]-1][0]
        sorted_cities[i][1] = cities[index[i][0]-1][1]
        sorted_cities[i][2] = cities[index[i][0]-1][2]
    return sorted_cities

def pseudo_euclidean_distance(point_1, point_2):
    
    x1 = point_1[0]
    y1 = point_1[1]
    x2 = point_2[0]
    y2 = point_2[1]
    
    xd = x1 - x2;
    yd = y1 - y2;
    rij = (((xd*xd + yd*yd)/10.0)**(1/2))
    tij = np.rint(rij);
    
    if (tij<rij):
        dij = tij + 1;
    else:
        dij = tij;
    
    return dij

## Main Function 

In [2]:
def Genetic_Algorithm(initial_route, polulation_size, elites, mutation_rate, generations):
    #generate out initial population of n number of individuals
    population = generate_population(initial_route, polulation_size)
    
    for gen in range (generations):
        population = create_next_generation(population, mutation_rate, elites)
         
    best_indv = population[0]
        
    return(best_indv)




## Generate Population

In [3]:

def generate_population(initial_route, polulation_size):
    #create a population of specified size by generating new individuals
    #and appending them to the population
    
    population = []
        
    for i in range(polulation_size):
        new_individual = generation_indv(initial_route)
        population.append(new_individual)
        
    return population

def generation_indv(initial_route):

        #number of genes in an individual (does not change)
        total_num_genes = len(initial_route)
        
        #generate new individual
        new_indivual = random.sample(initial_route,total_num_genes)
        
        return new_indivual


## Create "Next Generation" Function: Core of Genetic Algorthim

In [4]:
#Create the next generation

def create_next_generation(population, mutation_rate, elites):
    
    fitness_ranked_index = rank_population_fitness(population)
    selected_individuals_index = selection_process(elites,fitness_ranked_index)
    mating_pool = create_mating_pool(population, selected_individuals_index)
    children = breed_population(mating_pool, elites)
    next_generation = mutate_child_population(children, mutation_rate)
    
    return next_generation

    

## Create "Fitness Ranked Index"

In [5]:
# take all the individuals in a population and rank them by fitest first 
# fitness_ranked_index = rank_population_fitness(population)
def rank_population_fitness(population):
    fitness_index = []
    for i in range (len(population)):
        indiv_fitness_value = cal_individual_fitness(population[i])
        indv_ID = i
        fitness_index.append([indv_ID,indiv_fitness_value, 0, 0])

    #sort array fittest first of fitness ranked index (FRI)
    FRI = sort_fitness_index(fitness_index)
    
    #get cumulative sum of all fitness values
    cumulative_sum = 0
    for i in range (len(population)):  
        indiv_fitness_value = FRI[i][1] 
        
        cumulative_sum = cumulative_sum + indiv_fitness_value
        FRI[i][2] = cumulative_sum
    
    #cal cumulative percentage
    for i in range (len(population)):
        FRI[i][3] = (FRI[i][2]*100)/cumulative_sum
                                                  
    return FRI

def cal_individual_fitness(individual):
    route_distance = cal_route_distance(individual)
    individual_fitness_score =  (1 / route_distance)   
    return individual_fitness_score
 
def sort_fitness_index(unranked_fitness_index): 
    unranked_fitness_index.sort(key=lambda x: x[1],reverse = True)    
    return unranked_fitness_index
                                       

 ## Selection Process  
                                                    

In [6]:
def selection_process(elites_size,fitness_ranked_index):
    
    # Select Fittest individuals
    
    FRI = fitness_ranked_index
    selection_index = []
    
    #select the elites- append fittest idvs
    for i in range(elites_size):
        ID = FRI[i][0]
        selection_index.append(ID)
        
            
    for i in range(elites_size, len(FRI)):
        
        rand = 100*random.random()
    
        for i in range(0, len(FRI)):
            CFP = FRI[i][3]
            
            if CFP > rand:
                
                selection_index.append(FRI[i][0])
                break
        
    return selection_index
        

## Create the mating pool

In [7]:
def create_mating_pool(population, selected_individuals_index): 
                                                     
    # Extract the actual individuals from the population using the selected_individuals_index
    # and add them to the mating pool.

    mating_pool= []
    for i in range (len(selected_individuals_index)):
         
        individual_ID = selected_individuals_index[i]
        mating_pool.append(population[individual_ID])

                                                 
    return mating_pool


## Breed the population

In [8]:
def breed_population(mating_pool, elite_size):

    mating_pool_size = len(mating_pool) - elite_size
    all_children = []
    
    #the elites go forwards get a pass into new generation
    for i in range(0,elite_size):
        all_children.append(mating_pool[i]) 
    
    #breed the others
    for i in range(0, mating_pool_size):
        decrement = mating_pool_size-i-1
        child = breed(mating_pool[i], mating_pool[decrement])
        all_children.append(child)
    
    return all_children
        


def breed(parent_1, parent_2):
    
    #randomly find two genes 
    gene_A = int(random.random()*len(parent_1))
    gene_B = int(random.random()*len(parent_1))
    
    #find the begining and the end of the snipped section
    begin_gene = min(gene_A, gene_B)
    end_gene = max(gene_A, gene_B)
    
    spliced_section = []
    for i in range (begin_gene, end_gene):
        spliced_section.append(parent_1[i])
    
    
    remaining_genes = []
    for i in range (len(parent_2)):
        
        if (parent_2[i] not in spliced_section):
            
            remaining_genes.append(parent_2[i])
            
    child = spliced_section + remaining_genes
    return child
    

## Mutate Population

In [9]:
def mutate_child_population(population, mutation_rate):
    #For each individual in the population there is a chance of mutation
    mutant_population = []
    for indv in range (len(population)):
        mutant = mutate_individual(population[indv], mutation_rate)
        mutant_population.append(mutant)
        
    return mutant_population

def mutate_individual(individual, mutation_rate):  
    #A mutation in this case is where two genes in that individual will swap.
    mution_chance = random.random()

    if (mution_chance <= mutation_rate):
        # if mutation, first find  random city to swap with.
        random_gene_1 = random.sample(range(0, len(individual)), 1)
        random_gene_2 = random.sample(range(0, len(individual)), 1)
        mutant_individual = swap_genes(individual, random_gene_1, random_gene_2)
        
    else:
        mutant_individual = individual
   
    return mutant_individual

def swap_genes(individual, random_gene_1, random_gene_2):
        individual_copy = copy.copy(individual)
    
        gene_1 = individual_copy[random_gene_1[0]]
        gene_2 = individual_copy[random_gene_2[0]]
        #swap over (muation event)
        individual_copy[random_gene_1[0]] = gene_2
        individual_copy[random_gene_2[0]] = gene_1
               
        mutant = individual_copy
        return mutant
        


## Run Algorithm 

In [10]:
def cal_route_distance(route_solution): 
        
    Total_distance = 0
    
    new_route_sol = route_solution

    for i in range (len(new_route_sol)-1):
        
        x1 = new_route_sol[i][1]
        y1 = new_route_sol[i][2]
        x2 = new_route_sol[i+1][1]
        y2 = new_route_sol[i+1][2]
        
        point_1 = [x1, y1]
        point_2 = [x2, y2]
        
        Total_distance = Total_distance + pseudo_euclidean_distance(point_1, point_2)
        
    return Total_distance

#  Run

In [12]:

filename = "att48.tsp" 
initial_route = att48_to_nx3_list(filename) 
polulation_size = 100
elites = 20
mutation_rate = 0.01
generations = 300

best_route = Genetic_Algorithm(initial_route, polulation_size, elites, mutation_rate, generations)

print("best route dist ", cal_route_distance(best_route))


best route dist  12325.0
